### testing the importance of distance on instruction priority

In [38]:
%pip install huggingface_hub
%pip install python-dotenv
%pip install huggingface
%pip install datasets
%pip install transformers
%pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 297.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 195.1 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["DATASETS_DISABLE_PROGRESS_BAR"] = "1"

load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")

In [3]:
!hf auth login --token {huggingface_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `a3` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `a3`


In [39]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
#load wiki
ds = load_dataset("wikitext", "wikitext-2-raw-v1")

In [ ]:
print(type(ds))
print(ds)

<class 'datasets.dataset_dict.DatasetDict'>


In [17]:
print(ds["train"][7])

{'text': ' = = Gameplay = = \n'}


In [27]:
def keep_row(example):
    text = example['text']
    if not text:
        return False
    if text == "\n":
        return False
    # if text.startswith("="):
    #     return False
    return True

In [28]:
filtered_wiki = ds.filter(keep_row)

In [29]:
model_name = "google/gemma-2-9b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
### tokenize dataset
full_text = "\n".join([line for line in filtered_wiki["train"]["text"]])
all_tokens = tokenizer.encode(full_text, add_special_tokens = False)

In [ ]:
def get_r_wiki_slice(n_tokens):
    start_idx = random.randint(0, len(all_tokens) - n_tokens - 1)
    return all_tokens[start_idx : start_idx + n_tokens]

In [50]:
system_prompt = "Your goal is to respond to the user's request while making sure his instructions are fully satistfied. An instruction will appear in the form 'Instruction: <INSTRUCTION>.'\n"

end_question = "\nNow, provide your response based on the previous instructions. \n"

instruction_set = {
    "language": ["Respond in French.", "Respond in German."],
    "first_word": ["Begin your response with the word 'CHERRY'.", "Begin your response with the word 'APPLE'."],
}


In [44]:
def find_token_span(target_ids, full_ids):
    n = len(target_ids)
    for i in range(len(full_ids) - n + 1):
        if full_ids[i : i + n] == target_ids:
            return i, i + n
    return None, None

In [51]:
def construct_single_instr_prompt(instr_text, pos_pct, total_length=1024):
    formatted_instr = f"\n\nInstruction: {instr_text}\n\n"
    
    instr_tokens = tokenizer.encode(formatted_instr, add_special_tokens=False)
    sys_tokens = tokenizer.encode(system_prompt, add_special_tokens=False)
    end_tokens = tokenizer.encode(end_question, add_special_tokens=False)
    
    # calculate template overhead
    test_msg = [{"role": "user", "content": ""}]
    template_overhead = len(tokenizer.apply_chat_template(test_msg, tokenize=True, add_generation_prompt=True))
    
    # calculate filler budget
    fixed_len = len(sys_tokens) + len(instr_tokens) + len(end_tokens) + template_overhead
    filler_budget = total_length - fixed_len
    
    if filler_budget < 0:
        raise ValueError(f"Prompt too long ({fixed_len}) for target length ({total_length})")
        
    # get filler
    filler_tokens = get_r_wiki_slice(filler_budget)
    split_idx = int(pos_pct * filler_budget)
    
    # Decode only the wiki parts into strings
    pre_filler = tokenizer.decode(filler_tokens[:split_idx])
    post_filler = tokenizer.decode(filler_tokens[split_idx:])
    
    user_content = (
        system_prompt + 
        pre_filler + 
        formatted_instr + 
        post_filler + 
        end_question
    )
    
    messages = [{"role": "user", "content": user_content}]
    
    input_ids = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        return_tensors="pt"
    )
    
    full_list = input_ids[0].tolist()
    instr_indices = find_token_span(instr_tokens, full_list)
    
    if input_ids.shape[1] != total_length:
        if input_ids.shape[1] > total_length:
            input_ids = input_ids[:, :total_length]
        else:
            padding = torch.full((1, total_length - input_ids.shape[1]), tokenizer.pad_token_id)
            input_ids = torch.cat([input_ids, padding.to(input_ids.device)], dim=1)

    indices = {
        "instruction": instr_indices,
        "response_start": input_ids.shape[1] - 1
    }
    
    return input_ids, indices

In [52]:
target_word = " CHERRY" 
target_token_id = tokenizer.encode(target_word, add_special_tokens=False)[0]

In [ ]:
def batched_position_sweep(model, instr_text, total_length, samples_per_pos=100, batch_size=4):
    device = next(model.parameters()).device
    pcts = np.linspace(0.0, 1.0, 11)
    target_token_id = tokenizer.encode(" CHERRY", add_special_tokens=False)[0]
    
    results = {round(pct, 2): [] for pct in pcts}
    
    model.eval()
    
    for pct in pcts:
        print(f"Processing Position: {pct:.2f}")
        
        num_batches = (samples_per_pos + batch_size - 1) // batch_size
        
        for _ in tqdm(range(num_batches)):
            batch_list = []
            
            for _ in range(batch_size):
                input_ids, _ = construct_single_instr_prompt(instr_text, pct, total_length)
                input_ids = input_ids.to(device) 
                batch_list.append(input_ids)

            batch_tensor = torch.cat(batch_list, dim=0).to(device)
            
            with torch.no_grad():
                outputs = model(batch_tensor)

            last_token_logits = outputs.logits[:, -1, target_token_id]
            
            results[round(pct, 2)].extend(last_token_logits.cpu().tolist())
            
            del outputs
            torch.cuda.empty_cache()
            
    return results

In [56]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`

In [58]:
def plot_results(results):
    x = list(results.keys())
    means = [np.mean(results[pct]) for pct in x]
    stds = [np.std(results[pct]) for pct in x]
    
    plt.figure(figsize=(10, 6))
    plt.errorbar(x, means, yerr=stds, fmt='-o', capsize=5)
    plt.title("Instruction Met vs. Position (Gemma-2-9B)")
    plt.xlabel("Position in Haystack (0.0=Start, 1.0=End)")
    plt.ylabel("Logit of ' CHERRY'")
    plt.grid(True)
    plt.show()

# --- Execution ---
# Start with a shorter haystack to confirm the trend fast
sweep_data = batched_position_sweep(model, instruction_set["first_word"][0], total_length=2048, samples_per_pos=100)
plot_results(sweep_data)

Processing Position: 0.00


  0%|          | 0/25 [00:34<?, ?it/s]


KeyboardInterrupt: 

In [59]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Device count: {torch.cuda.device_count()}")

CUDA available: True
Device count: 1
